# pyRiemann-qiskit
## Qiskit Demo Day

In this notebook we will illustrates the use of pyRiemann-qiskit for the classification of EEG epochs (N x T).

The method presented here is called quantum-MDM. It uses classical MDM + convex model, optimized with QAOA.

### Setup

In [ ]:
!pip install pyRiemann-qiskit
!pip install moabb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.5/633.5 kB 38.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.9 MB/s eta 0:00:00
  Preparing me

In [ ]:
from pyriemann_qiskit.pipelines import (
    QuantumMDMWithRiemannianPipeline,
)
from moabb.datasets import BI2012
from moabb.paradigms import P300
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import LabelEncoder
from time import time

### Prepare data

In [ ]:
# Instantiate dataset
# This has to be adapted for the data you used
dataset = BI2012() #selected dataset
# Instantiate paradigm:
# It defines the pre-processing steps for data extraction
# This is specific to the MOABB library
paradigm = P300()
# Get first subject's data
X, y, _ = paradigm.get_data(dataset, subjects=[1])

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_function_transformer.py:310: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  return func(X, **(kw_args if kw_args else {}))


In [ ]:
# Encode class label into numeric value
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
# Let's keep it simple without cross-validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100, stratify=y)

### Classification

In [ ]:
# Instantiate our pipeline (inherits from sklearn mixins)
clf = QuantumMDMWithRiemannianPipeline(
    convex_metric="distance", quantum=True
)
display(clf._pipe)

In [ ]:
start_time = time()
clf.fit(X_train, y_train)
fit_time = time() - start_time

INFO:qiskit.utils.quantum_instance:
Qiskit Terra version: 0.45.0
Backend: 'aer_simulator_statevector (None)', with following setting:
{'basis_gates': ['ccx', 'cp', 'cswap', 'csx', 'cu', 'cu1', 'cu2', 'cu3', 'cx', 'cy', 'cz', 'delay', 'diagonal', 'ecr', 'h', 'id', 'initialize', 'mcp', 'mcphase', 'mcr', 'mcrx', 'mcry', 'mcrz', 'mcswap', 'mcsx', 'mcu', 'mcu1', 'mcu2', 'mcu3', 'mcx', 'mcx_gray', 'mcy', 'mcz', 'multiplexer', 'p', 'pauli', 'r', 'rx', 'rxx', 'ry', 'ryy', 'rz', 'rzx', 'rzz', 's', 'sdg', 'swap', 'sx', 'sxdg', 't', 'tdg', 'u', 'u1', 'u2', 'u3', 'unitary', 'x', 'y', 'z', 'break_loop', 'continue_loop', 'for_loop', 'if_else', 'kraus', 'qerror_loc', 'quantum_channel', 'roerror', 'save_amplitudes', 'save_amplitudes_sq', 'save_density_matrix', 'save_expval', 'save_expval_var', 'save_probabilities', 'save_probabilities_dict', 'save_state', 'save_statevector', 'save_statevector_dict', 'set_statevector', 'while_loop'], 'coupling_map': None}
{'initial_layout': None, 'seed_transpiler': 812

[QClass]  seed =  812191
[QClass]  Quantum simulation will be performed
GPU optimization disabled. No device found.
[QClass]  Fitting:  (614, 48, 48)
[QClass]  Feature dimension =  48
[QClass]  Convex MDM initiating algorithm
[QClass]  Using NaiveQAOAOptimizer
[QClass]  Training...


In [ ]:
# Perform classification (prediction)
# This use QAOA and a Aer/statevector simulator.
# An account token can also be provided to the estimator
# to run on real quantum backend
start_time = time()
y_pred = clf.predict(X_test)
pred_time = time() - start_time

[QClass]  Prediction:  (154, 48, 48)
[QClass]  Prediction finished.


### Results

In [ ]:
# Compute dataset balance
n_class_0 = y[y == 1].shape[0]
n_class_1 = y[y == 0].shape[0]

# Our dataset is not balanced
balance = n_class_0 / n_class_1
print(f'balance = {balance}')

balance = 0.2


In [ ]:
# Then we will use the balanced-accuracy
# which is a better metric than accuracy in this case:
score = balanced_accuracy_score(y_test, y_pred)
# Do not pay to much attention to the score.
# There is no cross-validation here.
# It likely just depends on the random_state.
print(f'score = {score}')

score = 0.65234375


In [ ]:
# Print execution time
# The fitting is classical, the prediction (here on simulator) is quantum.
# Other versions of the algorithm exist (with fitting quantum and prediction classical)
print("Fit time: {:.3f}s".format(fit_time))
print("Prediction time: {:.3f}s".format(pred_time))

Fit time: 0.813s
Prediction time: 82.011s


In [ ]:
# Time required for the classfication of a single trial:
n_train = y_train.shape[0]
n_test = y_test.shape[0]
print("Fit time (1 example): {:.3f}s".format(fit_time / n_train))
print("Prediction time (1 example): {:.3f}s".format(pred_time / n_test))

Fit time (1 example): 0.001s
Prediction time (1 example): 0.533s


### Work Direction


1.   Add more examples with different kind of datasets
2.   Test on real backend
3.   Better support for neural network

**Get involved :)**

### References

Github Repo: https://github.com/pyRiemann/pyRiemann-qiskit

API Documentation & Examples: https://pyriemann-qiskit.readthedocs.io

Paper: [pyRiemann-qiskit: A Sandbox for Quantum Classification Experiments with Riemannian Geometry](https://hal.science/hal-04040814/document)
